In [305]:
from serpapi import GoogleSearch
from pprint import pprint
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import json
import os

In [306]:
#########################################################
# WARNING!                                              #
# Don't run with this API key unless 100% required      #
#########################################################

# API Key Store
# Travis: 7b49b198668e629ad2ea004850f238d18e56e6e6f3ed19a4d71430a670f4ba29
# Leon: a50a83b24d1e51c42d6567fb1bd517dadd4d4348957fe3450928060bbb400ada
# Konni: 

params = {
    "engine": "google_jobs",
    "q": "data scientist",
    "hl": "en",
    "api_key": "a50a83b24d1e51c42d6567fb1bd517dadd4d4348957fe3450928060bbb400ada",
    "gl": "uk"
}

search = GoogleSearch(params)
results = search.get_dict()
jobs_results = results["jobs_results"]

https://serpapi.com/search


In [307]:
# Pretty Print the results
pprint(results)

{'chips': [{'options': [{'text': 'All'},
                        {'text': 'Data scientist', 'value': 'data scientist'},
                        {'text': 'Data science', 'value': 'data science'},
                        {'text': 'Engineer', 'value': 'engineer'},
                        {'text': 'Director', 'value': 'director'},
                        {'text': 'Manager', 'value': 'manager'},
                        {'text': 'Consultant', 'value': 'consultant'},
                        {'text': 'Principal', 'value': 'principal'},
                        {'text': 'Advisor', 'value': 'advisor'},
                        {'text': 'Analyst', 'value': 'analyst'},
                        {'text': 'Application engineer',
                         'value': 'application engineer'},
                        {'text': 'Associate', 'value': 'associate'},
                        {'text': 'Associate director',
                         'value': 'associate director'},
                        {'text': 'Data 

In [318]:
# Generate raw dataframe from scraped job listings
df_raw = pd.DataFrame.from_records(jobs_results)
df_raw.head(10)

,title,company_name,location,via,description,job_highlights,related_links,extensions,detected_extensions,job_id,thumbnail
0,"Associate Director, AI Research for Omics, Onc...",10001249 - Associate Principal Informatician,"Gaithersburg, MD, USA",via AstraZeneca Careers,This opportunity is available at our Gaithersb...,"[{'title': 'Qualifications', 'items': ['People...",[{'link': 'https://www.google.com/search?gl=uk...,"[2 days ago, Full–time]","{'posted_at': '2 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJBc3NvY2lhdGUgRGlyZWN0b3IsIE...,NaN
1,Data Scientist,FIS Global,"Cincinnati, OH, USA",via FIS Careers,Position Type :\nFull time\n...\nType Of Hire ...,"[{'title': 'Qualifications', 'items': ['Experi...","[{'link': 'http://www.fisglobal.com/', 'text':...","[2 days ago, Full–time]","{'posted_at': '2 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,Clinical Biomarker Data Scientist,Takeda Pharmaceutical,"Boston, MA, USA",via JobServe,"By clicking the Apply button, I understand tha...","[{'title': 'Qualifications', 'items': ['BS, MS...","[{'link': 'http://www.takeda.com/', 'text': 't...","[1 day ago, Full–time]","{'posted_at': '1 day ago', 'schedule_type': 'F...",eyJqb2JfdGl0bGUiOiJDbGluaWNhbCBCaW9tYXJrZXIgRG...,https://encrypted-tbn0.gstatic.com/images?q=tb...
3,Senior Data Scientist III,LexisNexis,Anywhere,via LinkedIn,• * Please note that we are open to hiring rem...,"[{'title': 'Qualifications', 'items': ['The id...",[{'link': 'http://www.lexisnexis.com/en-us/hom...,"[17 hours ago, Work from home, Full–time]","{'posted_at': '17 hours ago', 'schedule_type':...",eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,"Senior Data Scientist, Machine Learning: Platform",Rockstar Games,"New York, NY, USA (+1 other)",via The Muse,"At Rockstar Games, we create world-class enter...","[{'title': 'Qualifications', 'items': ['5+ yea...","[{'link': 'https://www.rockstargames.com/', 't...","[18 hours ago, Full–time]","{'posted_at': '18 hours ago', 'schedule_type':...",eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,https://encrypted-tbn0.gstatic.com/images?q=tb...
5,Data Scientist,Fidelity Investments Careers,"Westlake, TX, USA",via Fidelity Careers - Fidelity Investments,Job Description:\n\nThe Role...\n\nThe NextGen...,"[{'title': 'Qualifications', 'items': ['Deep u...","[{'link': 'http://www.fidelity.com/', 'text': ...","[5 days ago, Full–time]","{'posted_at': '5 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,https://encrypted-tbn0.gstatic.com/images?q=tb...
6,Data Scientist,Test,"New York, NY, USA",via Harnham,"Data Scientist\n\nNew York, New York...\n\n$12...","[{'title': 'Qualifications', 'items': ['A succ...",[{'link': 'https://www.google.com/search?gl=uk...,"[4 days ago, Full–time]","{'posted_at': '4 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,NaN
7,Senior Data Scientist - RCCB,Reyes Holdings,"Niles, IL, USA",via Careers With The Reyes Family Of Businesse...,Position Summary:\n\nThe Senior Data Scientist...,"[{'title': 'Responsibilities', 'items': ['The ...","[{'link': 'http://www.reyesholdings.com/', 'te...","[2 days ago, Full–time]","{'posted_at': '2 days ago', 'schedule_type': '...",eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,https://encrypted-tbn0.gstatic.com/images?q=tb...
8,Data Scientist IV,University of Florida,"Gainesville, FL, USA",via Inside Higher Ed Careers,Data Scientist IV\n\nJob No: 524105...\n\nWork...,"[{'title': 'Qualifications', 'items': ['Advanc...","[{'link': 'https://www.ufl.edu/', 'text': 'ufl...","[12 hours ago, Full–time]","{'posted_at': '12 hours ago', 'schedule_type':...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCBJViIsIm...,https://encrypted-tbn0.gstatic.com/images?q=tb...
9,Data Scientist,Merck,"Cambridge, MA, USA (+1 other)",via Merck Careers,Job Description\n\nThe Digital Operations

In [319]:
## DATA CLEANING ##

# Remove unnecessary bracket in Location
df_raw['location'] = df_raw['location'].str.replace('\([^()]*\)', "", regex=True)

# Remove via keyword from via
df_raw['via'] = df_raw['via'].str.replace('via', '', regex=False)

# Remove new line and other unnecessary characters
df_raw = df_raw.replace(to_replace=r'\n\n|\n•|\n', value=' ', regex=True)

# Remove unncecessary columns from the dataframe
df_raw.drop(columns=['thumbnail', 'extensions', 'related_links'], inplace=True)

In [315]:
## EXTRACTION ##

## Qualifications, responsibilities, and benefits ##
qualifications = []
responsibilities = []
benefits = []
highlights = []
def extract_inner_df(x): 
    for highlight in x:
        column_title = highlight.get('title')
        column_items = highlight.get('items')
        highlights_selection(column_title, column_items)
    equalSeries()
    
def highlights_selection(title, items):
    match title:
        case 'Qualifications':
            qualifications.append(items)
        case 'Responsibilities':
            responsibilities.append(items)
        case 'Benefits':
            benefits.append(items)
        case None:
            highlights.append(items)
            
def equalSeries():
    lengths = [len(qualifications), len(responsibilities), len(benefits), len(highlights)]
    max_length = max(lengths)
    
    if len(qualifications) < max_length:
        qualifications.append(None)
    if len(responsibilities) < max_length:
        responsibilities.append(None)
    if len(benefits) < max_length:
        benefits.append(None)
    if len(highlights) < max_length:
        highlights.append(None)
        
def merging_highlights():
    df_raw.drop(columns=['job_highlights'], inplace=True)
    df_raw['qualifications'] = qualifications
    df_raw['responsibilities'] = responsibilities
    df_raw['benefits'] = benefits
    df_raw['highlights'] = highlights
    
    
# Expand and merge job insights into the raw_df
df_raw['job_highlights'].apply(extract_inner_df)
merging_highlights()

## Posted_at, work from home, schedule type ##
extensions = pd.json_normalize(df_raw.detected_extensions)
df_raw.drop(columns=['detected_extensions'], inplace=True)
df_raw = pd.concat([df_raw, extensions], axis=1)

# Display
df_raw.head(10)

,title,company_name,location,via,description,job_id,qualifications,responsibilities,benefits,highlights,posted_at,schedule_type,work_from_home
0,"Data Scientist, Crypto Investment Firm",Recruiting from Scratch,"Plymouth, WA, USA",LinkedIn,Who is Recruiting from Scratch: Recruiting fro...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCwgQ3J5cH...,[1-2 years of product/strategy-oriented data s...,[The Data Scientist’s primary responsibility w...,[Starting salary of $150k with above-market be...,None,15 hours ago,Full–time,NaN
1,Informatics Data Science Engineer,Florida Cancer Specialists,"Tampa, FL, USA",JobServe,Job Description Summary: Under the direction o...,eyJqb2JfdGl0bGUiOiJJbmZvcm1hdGljcyBEYXRhIFNjaW...,"[Bachelor's degree in business analytics, comp...",[The Data Scientist works cross functionally w...,None,None,1 day ago,Full–time,NaN
2,Senior Data Scientist,Chubb INA Holdings Inc.,"New York, NY, USA",Chubb Careers,Company Description Chubb is the world’s large...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,"[Required Skills/Experience, 5+ years analytic...",[This role will work closely with engineering ...,"[In NYC the pay range for the role is $87,500 ...",None,4 days ago,Full–time,NaN
3,Data Scientist – US Commercial Data Science,Bristol Myers Squibb Careers,"Princeton, NJ, USA",Bristol Myers Squibb,Working with Us Challenging. Meaningful. Life-...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCDigJMgVV...,[Experience with cloud based environments (AWS...,"[In support of Commercialization Strategy, the...",None,None,3 days ago,Full–time,NaN
4,Data Scientist,Merck,"West Point, PA, USA",Merck Careers,Job Description The Digital Operations and Inn...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,[Masters degree with 3+ years' experience or P...,[DOI's mission is to advance non-clinical drug...,None,None,5 days ago,Full–time,NaN
5,Data Scientist,Fidelity Investments Careers,"Westlake, TX, USA",Fidelity Careers - Fidelity Investments,Job Description: The Role... The NextGen Core ...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,[Deep understanding of Machine Learning & Deep...,[Manage and guide data science and data engine...,"[$96,000 - $144,000 per year, This position is...",None,3 days ago,Full–time,NaN
6,Principal Data Scientist,Overstory,Anywhere,AngelList,About the role We are looking for a Principal ...,eyJqb2JfdGl0bGUiOiJQcmluY2lwYWwgRGF0YSBTY2llbn...,None,None,None,[About the role\n\nWe are looking for a Princi...,3 days ago,Full–time,True
7,"Data Scientist, Lead Jobs",Booz Allen Hamilton,"Las Vegas, NV, USA",Clearance Jobs,"Job Number: R0165196 Data Scientist, Lead... T...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCwgTGVhZC...,"[6+ years of experience with data science, dat...",[You'll work closely with your customer to und...,[With professional and leadership development ...,None,2 days ago,Full–time,NaN
8,Data Scientist (Forecasting) (US),Tiger Analytics,Anywhere,Startup Jobs,Tiger Analytics is looking for experienced Dat...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAoRm9yZW...,"[Master’s or Ph.D. in Statistics, Economics, D...",[Effectively communicate the analytics approac...,[This position offers an excellent opportunity...,None,3 days ago,Full–time,True
9,Sr. Data Scientist (NLP),MCKESSON,"Texas, USA",Jobs At MCKESSON,Ontada is a leading oncology real-world data a...,eyJqb2JfdGl0bGUiOiJTci4gRGF0YSBTY2llbnRpc3QgKE...,[5+ years of industry experience in ML and/or ...,"[Collaborate with product management, product ...","[As part of Total Rewards, we are proud to off...",None,2 days ago,Full–time,NaN


In [316]:
## VALUE MANIPULATION ##

# Convert work from home into binary encoding
df_raw['work_from_home'].replace({True: 1, np.NaN: 0}, inplace=True)

# Posted_on conversion to date
def date_finder(delta):
    current_date = datetime.today()
    date_format = '%Y-%m-%d'
    if day_key(delta):
        delta_days = re.findall(r'\d+', delta)
        delta_days = int(delta_days[0])
        posted_date = current_date - timedelta(days=delta_days)
        return posted_date.strftime(date_format)
    return current_date.strftime(date_format)
    
def day_key(observed_on_str):
    terms = observed_on_str.split()
    if 'day' in terms:
        return True
    return False

# Apply date_finder
df_raw['posted_at'] = df_raw['posted_at'].apply(date_finder)

# Display resulting dataframe
df_raw.head(10)

,title,company_name,location,via,description,job_id,qualifications,responsibilities,benefits,highlights,posted_at,schedule_type,work_from_home
0,"Data Scientist, Crypto Investment Firm",Recruiting from Scratch,"Plymouth, WA, USA",LinkedIn,Who is Recruiting from Scratch: Recruiting fro...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCwgQ3J5cH...,[1-2 years of product/strategy-oriented data s...,[The Data Scientist’s primary responsibility w...,[Starting salary of $150k with above-market be...,None,2023-02-28,Full–time,0.0
1,Informatics Data Science Engineer,Florida Cancer Specialists,"Tampa, FL, USA",JobServe,Job Description Summary: Under the direction o...,eyJqb2JfdGl0bGUiOiJJbmZvcm1hdGljcyBEYXRhIFNjaW...,"[Bachelor's degree in business analytics, comp...",[The Data Scientist works cross functionally w...,None,None,2023-02-27,Full–time,0.0
2,Senior Data Scientist,Chubb INA Holdings Inc.,"New York, NY, USA",Chubb Careers,Company Description Chubb is the world’s large...,eyJqb2JfdGl0bGUiOiJTZW5pb3IgRGF0YSBTY2llbnRpc3...,"[Required Skills/Experience, 5+ years analytic...",[This role will work closely with engineering ...,"[In NYC the pay range for the role is $87,500 ...",None,2023-02-28,Full–time,0.0
3,Data Scientist – US Commercial Data Science,Bristol Myers Squibb Careers,"Princeton, NJ, USA",Bristol Myers Squibb,Working with Us Challenging. Meaningful. Life-...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCDigJMgVV...,[Experience with cloud based environments (AWS...,"[In support of Commercialization Strategy, the...",None,None,2023-02-28,Full–time,0.0
4,Data Scientist,Merck,"West Point, PA, USA",Merck Careers,Job Description The Digital Operations and Inn...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,[Masters degree with 3+ years' experience or P...,[DOI's mission is to advance non-clinical drug...,None,None,2023-02-28,Full–time,0.0
5,Data Scientist,Fidelity Investments Careers,"Westlake, TX, USA",Fidelity Careers - Fidelity Investments,Job Description: The Role... The NextGen Core ...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCIsImh0aW...,[Deep understanding of Machine Learning & Deep...,[Manage and guide data science and data engine...,"[$96,000 - $144,000 per year, This position is...",None,2023-02-28,Full–time,0.0
6,Principal Data Scientist,Overstory,Anywhere,AngelList,About the role We are looking for a Principal ...,eyJqb2JfdGl0bGUiOiJQcmluY2lwYWwgRGF0YSBTY2llbn...,None,None,None,[About the role\n\nWe are looking for a Princi...,2023-02-28,Full–time,1.0
7,"Data Scientist, Lead Jobs",Booz Allen Hamilton,"Las Vegas, NV, USA",Clearance Jobs,"Job Number: R0165196 Data Scientist, Lead... T...",eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCwgTGVhZC...,"[6+ years of experience with data science, dat...",[You'll work closely with your customer to und...,[With professional and leadership development ...,None,2023-02-28,Full–time,0.0
8,Data Scientist (Forecasting) (US),Tiger Analytics,Anywhere,Startup Jobs,Tiger Analytics is looking for experienced Dat...,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCAoRm9yZW...,"[Master’s or Ph.D. in Statistics, Economics, D...",[Effectively communicate the analytics approac...,[This position offers an excellent opportunity...,None,2023-02-28,Full–time,1.0
9,Sr. Data Scientist (NLP),MCKESSON,"Texas, USA",Jobs At MCKESSON,Ontada is a leading oncology real-world data a...,eyJqb2JfdGl0bGUiOiJTci4gRGF0YSBTY2llbnRpc3QgKE...,[5+ years of industry experience in ML and/or ...,"[Collaborate with product management, product ...","[As part of Total Rewards, we are proud to off...",None,2023-02-28,Full–time,0.0


In [317]:
def write_data():
    file_name = 'interim_data.csv'
    data_exists = os.path.isfile(file_name)
    if data_exists:
        df_raw.to_csv(file_name, mode='a', index=False, header=False)
    else:
        df_raw.to_csv(file_name, mode='w', index=False, header=True)
        
write_data()